### 옷 분리
    
    최초 작성일 : 20/03/09
    작성자 : 양희승
    
    작성내용 : 무신사 이미지를 통해 옷 색깔 판별하기 위한 옷분리 코드
    
    
    수정내용 :
        전체 코드 수정
        
        20/03/20
            - 분리코드 수정
            - DB 적재 코드 추가
            - 옷 색 판별 코드 추가
            
        20/03/23
            - 분리코드 수정  이미지 RGBA -> 투명배경 흰색 처리
            - DB연결

In [131]:
import pymysql
import cv2 as cv
import numpy as np
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import tensorflow as tf
import sys
import os
import time
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

d:\users\yhs\appdata\local\programs\python\python36\lib\site-packages\tensorflow_core\python\client\session.py:1752: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [132]:
# ## DB 연결 데이터 
# config = {"host":"192.168.0.12", "user":"cai", "passwd":"1234", "db":"final"}
config = {"host":"49.142.181.65", "user":"cai","passwd":"1234", "db":"final"}
conn = pymysql.connect(**config)

# 모델, file name저장
files = os.listdir("Fashion-AI/input/")
model = load_model("Fashion-AI/model/topwears.h5")

In [172]:
#  옷 분할

class fashion_tools(object):
    def __init__(self,imageid,version=1.1):
        self.imageid = imageid
        self.model = model
        self.version = version
        
    def get_dress(self,stack=False):
        name =  self.imageid
        file = cv.imread(name)
        file = tf.image.resize_with_pad(file,target_height=512,target_width=512)
        rgb  = file.numpy()
        file = np.expand_dims(file,axis=0)/ 255.
        seq = self.model.predict(file)
        seq = seq[3][0,:,:,0]
        seq = np.expand_dims(seq,axis=-1)
        c1x = rgb*seq
        c2x = rgb*(1-seq)
        cfx = c1x+c2x
        dummy = np.ones((rgb.shape[0],rgb.shape[1],1))
        rgbx = np.concatenate((rgb,dummy*255),axis=-1)
        rgbs = np.concatenate((cfx,seq*255.),axis=-1)
        if stack:
            stacked = np.hstack((rgbx,rgbs))
            return stacked
        else:
            alpha_val = np.zeros((512,512))
            result = np.array(rgbs[:,:,0:3])
            for i in range(len(rgbs[:,:,3])) :
                for j in range(len(rgbs[:,:,3][i])):
                    alpha_val[i][j] = round(rgbs[:,:,3][i][j], 2)
            
            for i in range(len(alpha_val)) :
                for j in range(len(alpha_val[i])) :
                    if (alpha_val[i][j] <= 0.95) :
                        result[i][j][0] = 255.0
                        result[i][j][1] = 255.0
                        result[i][j][2] = 255.0                        
            return result
        
#     def get_patch(self):
#         return None
    
#     def development(self):
#         n_ = '\n'
#         return (f"VERSION : {self.version} {n_} Interesting tools to be added to the workflow pipe!!!")

In [3]:
idx = int(input("숫자만 입력 : "))

숫자만 입력 : 1


In [155]:
start_time = time.time()

for i in range(idx) :
    
#     f = "Fashion-AI/input/"+files[i]
    f = "Fashion-AI/input/27651.jpg"
    o = "Fashion-AI/output/"+files[i].replace(".jpg", "")
    api    = fashion_tools(f, model)    
    image_ = api.get_dress(False)    
    
end_time = time.time()
print("total time : ", end_time - start_time, "초")

error: OpenCV(4.2.0) C:\projects\opencv-python\opencv\modules\imgproc\src\thresh.cpp:1529: error: (-215:Assertion failed) src.type() == CV_8UC1 in function 'cv::threshold'


In [173]:
f = "Fashion-AI/input/27651.jpg"
o = "Fashion-AI/output/"+files[i].replace(".jpg", "")
api    = fashion_tools(f, model)    
image_ = api.get_dress(False)

In [134]:
# 배경색처리 - 8번 오류
# cv2.namedWindow(‘image’, cv2.WINDOW_NORMAL)

def kim(img) :
#     img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    imgo = img
    height, width = imgo.shape[:2]

    #Create a mask holder
    mask = np.zeros(imgo.shape[:2],np.uint8)

    #Grab Cut the object
    bgdModel = np.zeros((1,65),np.float64)
    fgdModel = np.zeros((1,65),np.float64)

    #Hard Coding the Rect… The object must lie within this rect.
    rect = (10,10,width-30,height-30)
    cv.grabCut(imgo,mask,rect,bgdModel,fgdModel,5,cv.GC_INIT_WITH_RECT)
    mask = np.where((mask==2)|(mask==0),0,1).astype("uint8")
    img1 = imgo*mask[:,:,np.newaxis]

    #Get the background
    background = imgo-img1

    #Change all pixels in the background that are not black to white
    background[np.where((background > [0,0,0]).all(axis = 2))] = [255,255,255]

    #Add the background and the image
    final = background + img1

    #To be done – Smoothening the edges….

    # plt.imshow("image", final )
#     cv.imwrite("img/clothes_test/ext_"+file_name+".jpg", final)
#     plt.imshow(final)
#     plt.show()
    return final

In [150]:
## 경계값 처리

def test(img) :
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    ret, thresh = cv.threshold(gray, 0, 255, cv.THRESH_BINARY_INV+cv.THRESH_OTSU)


    kernel = np.ones((3,3), np.uint8)
    opening = cv.morphologyEx(thresh, cv.MORPH_OPEN, kernel, iterations=2)

    sure_bg = cv.dilate(opening, kernel, iterations=3)

    dist_transform = cv.distanceTransform(opening, cv.DIST_L2, 5)

    ret, sure_fg = cv.threshold(dist_transform, 0.5*dist_transform.max(), 255,0)
    sure_fg = np.uint8(sure_fg)

    unknown = cv.subtract(sure_bg, sure_fg)

    ret, markers = cv.connectedComponents(sure_fg)
    markers = markers+1

    markers[unknown == 255] = 0


    markers = cv.watershed(img, markers)
    img[markers == -1] = [ 255, 0 , 0]

    images = [gray, thresh, sure_g, dist_transform, sure_fg, unknown, markers, img]

    titles = ['Gray', 'Binary', 'Sure BG', 'Distance', 'Sure FG', 'Unknown', 'Markers', 'Result']

    for i in range(len(images)) :
        plt.subplot(2,4,i+1), plt.imshow(images[i]), plt.title(titles[i]), plt.xticks([]), plt.yticks([])

    plt.show()

In [178]:
img = image_/255

mask = np.zeros(img.shape[:2],np.uint8)

bgdModel = np.zeros((1,65),np.float64)
fgdModel = np.zeros((1,65),np.float64)

rect = (161,79,150,150)

cv.grabCut(img,mask,rect,bgdModel,fgdModel,5,cv.GC_INIT_WITH_RECT)
mask2 = np.where((mask==2)|(mask==0),0,1).astype('uint8')
img = img*mask2[:,:,np.newaxis]

plt.imshow(img)
plt.colorbar()
plt.show()

error: OpenCV(4.2.0) C:\projects\opencv-python\opencv\modules\imgproc\src\grabcut.cpp:557: error: (-5:Bad argument) image must have CV_8UC3 type in function 'cv::grabCut'


In [135]:
result = kim(image_)
plt.imshow(result)
plt.show()

error: OpenCV(4.2.0) C:\projects\opencv-python\opencv\modules\imgproc\src\grabcut.cpp:557: error: (-5:Bad argument) image must have CV_8UC3 type in function 'cv::grabCut'
